In [1]:
!pip install torch torchvision opencv-python 
!pip install Pillow scikit-learn cupy 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.3 MB/s eta 0:00:00 MB/s eta 0:00:0101
  Preparing metadata (setup.py) ... done
  Using cached fastrlock-0.8.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl (51 kB)
  Created wheel for cupy: filename=cupy-13.1.0-cp310-cp310-linux_x86_64.whl size=88744733 sha256=a41df0675493ba6c4a33171f7d9d88115a7d848a1442b3a74f92a0096f5b59fc
  Stored in directory: /home/roman/.cache/pip/wheels/3c/f7/fd/f440432bc00ac2dd12e25ed1b47e30b3fb6744fb46eebcc066
Successfully built cupy


In [2]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
import zipfile
import numpy as np
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator
from scipy.signal import find_peaks, peak_widths
from PIL import Image
import gc
from joblib import Parallel, delayed

In [3]:
model = resnet50(pretrained=True)
model.eval()

# Преобразования для входных данных модели
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Select the desired layer
layer = model._modules.get('avgpool')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




/home/roman/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/roman/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:

def extract_frame_embedding(frame, model, preprocess, device):
    input_tensor = preprocess(frame).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(input_tensor)
    return embedding

def extract_frame_embeddings(video_path, model, preprocess, device, layer, frame_interval=None):
    # Открытие видеофайла
    outputs = []

    # attach hook to the penultimate layer
    def copy_embeddings(m, i, o):
        """Copy embeddings from the penultimate layer."""
        o = o[:, :, 0, 0].detach().cpu().squeeze().numpy().tolist()
        outputs.append(o)

    _ = layer.register_forward_hook(copy_embeddings)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if frame_interval is None:
        frame_interval = int(fps)
    print(f"Framerate is {frame_interval}")

    frames = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frames.append(frame)

        frame_count += 1

    cap.release()

    # Параллельная обработка кадров
    for frame in frames:
        _ = extract_frame_embedding(frame, model, preprocess, device)

    return np.array(outputs)


def extract_videos_from_zip(zip_path, filename, model, preprocess, device, layer ):
    embeddings_dict = {}

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        video_files = [f for f in zip_ref.namelist() if f.endswith(('.mp4', '.avi', '.mov'))]
        for video_file in video_files:
            if filename not in video_file:
                continue
            print(f"Processing {video_file}...")

            with zip_ref.open(video_file) as video_stream:

                # Чтение видео из байтового потока
                video_data = video_stream.read()
                video_array = np.frombuffer(video_data, np.uint8)


                # Временный файл для OpenCV VideoCapture
                temp_video_path = 'temp_video.mp4'
                with open(temp_video_path, 'wb') as temp_video_file:
                    temp_video_file.write(video_data)

                # Извлечение эмбеддингов из видео
                embeddings = extract_frame_embeddings(temp_video_path, model, preprocess, device, layer)
                #embeddings = extract_frame_embeddings_lib(temp_video_path)
                embeddings_dict[video_file] = embeddings

    return embeddings_dict

In [5]:
df = pd.read_csv("/media/roman/Transcend/rutube/piracy_val.csv")

In [6]:
df

,Unnamed: 0,ID_piracy,segment,ID_license,segment.1
0,0,ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4,1539-1685,ded3d179001b3f679a0101be95405d2c.mp4,546-692
1,1,t9j5gg42w6s7f62uybhtc51aj5nq9vxd.mp4,158-222,f4b1fd188fe77f9f56de07e867128b13.mp4,392-456
2,2,zui6ud3solfndyetlralstpcjmv3br1f.mp4,139-263,aa16953bbb8fdb2dfc5fc368f4abe89a.mp4,656-780
3,3,aozskym3m7s9ibw5xpgca7m5jqe240vs.mp4,306-449,3219f476e4bc29f6f420e4e6b05ae8bb.mp4,399-542
4,4,3y78kqwdr5lbtsy9tb76n5ylbcrainsx.mp4,517-590,cb718f23524eb08af2f8036eedc6b50c.mp4,119-192
...,...,...,...,...,...
133,133,4ojku252jnbtv04wtnvm73x08xyftpgy.mp4,466-632,fd316c0e150c30ba2156f82e65bfea62.mp4,322-488
134,134,iflvtrjcx50j1kgbb3zgbvpql6zm2gry.mp4,196-471,a808f0de343faf6320bcc94ea57c5381.mp4,462-737
135,135,ztbk9ua8hx1u6l3ol3q5ehh8ugxqlrpd.mp4,165-213,ce9697edebdbec509944e8a010d91d0e.mp4,59-107
136,136,dtpq232karpm09ldxucjqip32hfblrkz.mp4,98-231,8cc8785efca469cdb6fb6efdcd3421d2.mp4,227-360


In [40]:
row_test = 79
def get_piracy_lic_embeddings(piracy_id, license_id):

    zip_path = '/media/roman/Transcend/rutube/index.zip'
    embeddings_dict_license = extract_videos_from_zip(zip_path, license_id, model, preprocess, device, layer)
    zip_path = '/media/roman/Transcend/rutube/val.zip'
    embeddings_dict_piracy = extract_videos_from_zip(zip_path, piracy_id, model, preprocess, device, layer)
    embeddings_dict = dict(list(embeddings_dict_license.items()) + list(embeddings_dict_piracy.items()))
    license_np = np.asarray(list(embeddings_dict.values())[0], dtype="float64")
    piracy_np = np.asarray(list(embeddings_dict.values())[1], dtype="float64")
    return  piracy_np,license_np


In [41]:
def cosine_distance_gpu(A, B):
    # Убедимся, что входные данные являются cupy массивами
    if not isinstance(A, cp.ndarray):

        A = cp.array(A)
    if not isinstance(B, cp.ndarray):
        B = cp.array(B)
    
    dot_product = cp.dot(A, B.T)
    norm_A = cp.linalg.norm(A, axis=1, keepdims=True)
    norm_B = cp.linalg.norm(B, axis=1, keepdims=True)
    return 1 - dot_product / (norm_A * norm_B.T)

def gpu_cosine_distance_matrix(matrix1, matrix2):
    # Убедимся, что входные данные являются cupy массивами
    if not isinstance(matrix1, cp.ndarray):
        matrix1 = cp.array(matrix1)
    if not isinstance(matrix2, cp.ndarray):
        matrix2 = cp.array(matrix2)
    
    # Вычисляем косинусное расстояние между всеми парами строк матриц
    distance_matrix = cosine_distance_gpu(matrix1, matrix2)
    # Преобразуем cupy array в numpy array
    distance_matrix = distance_matrix.get()
    return distance_matrix

In [42]:
piracy =  df.iloc[row_test]["ID_piracy"]
license = df.iloc[row_test]["ID_license"]
segment_piracy = df.iloc[row_test]["segment"]
segment_lic = df.iloc[row_test]["segment.1"]
print (f"Pair of license {license} and pirate {piracy} has interval {segment_piracy} and {segment_lic}")
piracy_np, license_np = get_piracy_lic_embeddings(piracy, license)

Pair of license afa4ac13ec1628e7f66493783cf7550e.mp4 and pirate wd6i4pislndwtth1hc0yrg23f9y3r8gw.mp4 has interval 53-134 and 642-723
Processing compressed_index/afa4ac13ec1628e7f66493783cf7550e.mp4...
Framerate is 10
Processing compressed_val/wd6i4pislndwtth1hc0yrg23f9y3r8gw.mp4...
Framerate is 10


In [43]:
license_np.shape

(1441, 2048)

In [44]:
piracy_np.shape

(379, 2048)

In [45]:
type(license_np)

numpy.ndarray

In [46]:
matrix = gpu_cosine_distance_matrix(piracy_np, license_np)

<class 'numpy.ndarray'>


/home/roman/.local/lib/python3.10/site-packages/cupy/cuda/compiler.py:233: PerformanceWarning: Jitify is performing a one-time only warm-up to populate the persistent cache, this may take a few seconds and will be improved in a future release...
  jitify._init_module()


In [14]:
def make_plt_rows(matrix_l):
    points_dict = {}

    # Проходимся по строкам матрицы косинусных расстояний
    for i in range(max(matrix_l.shape[0], matrix_l.shape[1])):
        points_dict[i] = 0
    max_value_global = np.max(np.abs(matrix_l))
    mean_value_global = np.mean(np.abs(matrix_l))
    for i in range(matrix_l.shape[0]):
        min_index = np.argmin(matrix_l[i])
        min_value = matrix_l[i, min_index]


        # Вычисляем y-координату для точки на графике
        y_value = max_value_global - min_value

        if (min_value < mean_value_global):
            points_dict[min_index] =y_value

    # Преобразуем словарь в списки для построения графика
    x_points = list(points_dict.keys())
    y_points = list(points_dict.values())
    # Убедимся, что все значения y положительны
    y_points = np.maximum(y_points, 0)

    sorted_indices = np.argsort(x_points)
    print(max(x_points))
    x_points = np.take(np.array(x_points),sorted_indices)
    y_points = np.take(np.array(y_points),sorted_indices)
    print(max(x_points))
    def moving_average(data, window_size):
        return np.convolve(data, np.ones(window_size)/window_size, mode='same')

    window_size = max(int(y_points.shape[0] * 0.05), 3)
    y_points_smoothed = moving_average(y_points, window_size)

    f_interp = PchipInterpolator(x_points, y_points_smoothed)
    x_smooth = np.linspace(x_points.min(), x_points.max(), len(x_points))
    y_smooth = f_interp(x_smooth)

    peaks, _ = find_peaks(y_smooth)
    widths_half_max = peak_widths(y_smooth, peaks, rel_height=0.50)

    max_peak_idx = np.argmax(y_smooth[peaks])
    max_peak_height = y_smooth[peaks][max_peak_idx]

    max_width_idx = np.argmax(widths_half_max[0])
    max_peak_width = widths_half_max[0][max_width_idx]
    left_ips_x = 0
    right_ips_x = 0
    if peaks[max_peak_idx] == peaks[max_width_idx]:
        print(f"Пик одновременно самый высокий и самый широкий: высота={max_peak_height}, ширина={max_peak_width}")

        left_ips_x = x_smooth[int(widths_half_max[2][max_width_idx])]
        right_ips_x = x_smooth[int(widths_half_max[3][max_width_idx])]

        print(f"Границы пика относительно оси x: {left_ips_x}, {right_ips_x}")
    elif peaks[max_peak_idx]: #and widths_half_max[0][max_peak_idx] > 50:
        left_ips_x = x_smooth[int(widths_half_max[2][max_peak_idx])]
        right_ips_x = x_smooth[int(widths_half_max[3][max_peak_idx])]

        print(f"Границы пика относительно оси x: {left_ips_x}, {right_ips_x}")
    # plt.plot(x_smooth, y_smooth)
    # plt.xlabel('Index of Minimum Cosine Distance')
    # plt.ylabel('Sum of Max Value - Min Value')
    # plt.title('Graph of Minimum Cosine Distances with Peaks')
    # plt.grid(True)
    # plt.show()
    return f"{left_ips_x}-{right_ips_x}"



In [15]:
def make_plt_columns(matrix_l):
    points_dict = {}
    for j in range(max(matrix_l.shape[0], matrix_l.shape[1])):
        points_dict[j] = 0
    print(matrix_l.shape[1])
    max_value_global = np.max(np.abs(matrix_l))
    mean_value_global = np.mean(np.abs(matrix_l))
    # Проходимся по строкам матрицы косинусных расстояний
    for j in range(matrix_l.shape[1]):
        min_index = np.argmin(matrix_l[:, j])
        min_value = matrix_l[min_index, j]

        # Вычисляем y-координату для точки на графике
        y_value = max_value_global - min_value

        if (min_value < mean_value_global):
            points_dict[min_index] =y_value

    x_points = list(points_dict.keys())
    y_points = list(points_dict.values())


    sorted_indices = np.argsort(x_points)
    print(max(x_points))
    x_points = np.take(np.array(x_points),sorted_indices)
    y_points = np.take(np.array(y_points),sorted_indices)
    print(max(x_points))
    def moving_average(data, window_size):
        return np.convolve(data, np.ones(window_size)/window_size, mode='same')

    window_size = max(int(y_points.shape[0] * 0.05), 3)
    y_points_smoothed = moving_average(y_points, window_size)

    f_interp = PchipInterpolator(x_points, y_points_smoothed)
    x_smooth = np.linspace(x_points.min(), x_points.max(), len(x_points))
    y_smooth = f_interp(x_smooth)

    peaks, _ = find_peaks(y_smooth)
    widths_half_max = peak_widths(y_smooth, peaks, rel_height=0.50)

    max_peak_idx = np.argmax(y_smooth[peaks])
    max_peak_height = y_smooth[peaks][max_peak_idx]

    max_width_idx = np.argmax(widths_half_max[0])
    max_peak_width = widths_half_max[0][max_width_idx]
    left_ips_x = 0
    right_ips_x = 0
    if peaks[max_peak_idx] == peaks[max_width_idx]:
        print(f"Пик одновременно самый высокий и самый широкий: высота={max_peak_height}, ширина={max_peak_width}")

        left_ips_x = x_smooth[int(widths_half_max[2][max_width_idx])]
        right_ips_x = x_smooth[int(widths_half_max[3][max_width_idx])]

        print(f"Границы пика относительно оси x: {left_ips_x}, {right_ips_x}")
    elif peaks[max_peak_idx]: #and widths_half_max[0][max_peak_idx] > 50#:
        left_ips_x = x_smooth[int(widths_half_max[2][max_peak_idx])]
        right_ips_x = x_smooth[int(widths_half_max[3][max_peak_idx])]

        print(f"Границы пика относительно оси x: {left_ips_x}, {right_ips_x}")

    # plt.plot(x_smooth, y_smooth)
    # plt.xlabel('Index of Minimum Cosine Distance')
    # plt.ylabel('Sum of Max Value - Min Value')
    # plt.title('Graph of Minimum Cosine Distances with Peaks')
    # plt.grid(True)
    # plt.show()
    return f"{left_ips_x}-{right_ips_x}"



In [16]:
print(matrix.shape)

(379, 1441)


In [17]:
type(matrix[0][0])

numpy.float64

In [18]:

make_plt_rows(matrix)

1440
1440
Границы пика относительно оси x: 644.0, 725.0


'644.0-725.0'

In [19]:
make_plt_columns(matrix)

1441
1440
1440
Границы пика относительно оси x: 49.0, 130.0


/tmp/ipykernel_44336/3498212911.py:39: PeakPropertyWarning: some peaks have a width of 0
  widths_half_max = peak_widths(y_smooth, peaks, rel_height=0.50)


'49.0-130.0'

In [20]:
def get_intervals(id_piracy, id_license):
    piracy_l ,license_l= get_piracy_lic_embeddings(id_piracy, id_license)
    matrix_l = gpu_cosine_distance_matrix(license_l, piracy_l)
    result = make_plt_rows(matrix_l), make_plt_columns(matrix_l)
    del matrix_l
    del license_l
    del piracy_l
    gc.collect()
    return result


def parse_segment(segment):
    start, end = map(float, segment.split("-"))
    return start, end


def iou(segment_q, segment_t):
    start_q, stop_q = parse_segment(segment_q)
    start_t, stop_t = parse_segment(segment_t)

    intersection_start = max(start_q, start_t)
    intersection_end = min(stop_q, stop_t)

    intersection_length = max(0, intersection_end - intersection_start)
    union_length = (stop_q - start_q) + (stop_t - start_t) - intersection_length

    iou = intersection_length / union_length if union_length > 0 else 0
    return iou



# Вычисляем метрики IOU и их произведение без добавления промежуточных колонок
def calculate_iou_product(row):
    piracy_l =  row["ID_piracy"]
    license_l = row["ID_license"]
    segment_piracy_l = row["segment"]
    segment_lic_l = row["segment.1"]
    print(f"Pair of priracy {piracy_l} and license {license_l} has interval {segment_piracy_l} and {segment_lic_l}")
    interval_piracy, interval_license = get_intervals(piracy_l, license_l)
    iou_piracy = iou(segment_piracy_l, interval_piracy)
    iou_license = iou(segment_lic_l, interval_license)
    print(f"{interval_piracy} | {interval_license} IOU p {iou_piracy} IOU l {iou_license}")
    return iou_piracy * iou_license



In [21]:
df['iou_product'] = df.apply(calculate_iou_product, axis=1)

Pair of priracy ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4 and license ded3d179001b3f679a0101be95405d2c.mp4 has interval 1539-1685 and 546-692
Processing compressed_index/ded3d179001b3f679a0101be95405d2c.mp4...
Framerate is 10
Processing compressed_val/ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4...
Framerate is 10
1777
1777
Границы пика относительно оси x: 1521.0, 1677.0
1778
1777
1777
Пик одновременно самый высокий и самый широкий: высота=0.26046930988092226, ширина=136.28469328817891
Границы пика относительно оси x: 1167.0, 1303.0


/tmp/ipykernel_44336/464454489.py:42: PeakPropertyWarning: some peaks have a width of 0
  widths_half_max = peak_widths(y_smooth, peaks, rel_height=0.50)
/tmp/ipykernel_44336/3498212911.py:39: PeakPropertyWarning: some peaks have a width of 0
  widths_half_max = peak_widths(y_smooth, peaks, rel_height=0.50)


1521.0-1677.0 | 1167.0-1303.0 IOU p 0.8414634146341463 IOU l 0.0
Pair of priracy t9j5gg42w6s7f62uybhtc51aj5nq9vxd.mp4 and license f4b1fd188fe77f9f56de07e867128b13.mp4 has interval 158-222 and 392-456
Processing compressed_index/f4b1fd188fe77f9f56de07e867128b13.mp4...
Framerate is 10
Processing compressed_val/t9j5gg42w6s7f62uybhtc51aj5nq9vxd.mp4...
Framerate is 10


KeyboardInterrupt: 

In [ ]:
import cupy as cp

def cosine_distance_gpu(A, B):
    dot_product = cp.dot(A, B)
    norm_A = cp.linalg.norm(A)
    norm_B = cp.linalg.norm(B)
    return 1 - dot_product / (norm_A * norm_B)

def gpu_cosine_distance_matrix(matrix1, matrix2):
    rows_matrix1 = matrix1.shape[0]
    rows_matrix2 = matrix2.shape[0]
    # Создаем матрицу для хранения косинусных расстояний на GPU
    distance_matrix = cp.zeros((rows_matrix1, rows_matrix2))
    for i in range(rows_matrix1):
        for j in range(rows_matrix2):
            distance_matrix[i, j] = cosine_distance_gpu(matrix1[i], matrix2[j])
    # Преобразуем cupy array в numpy array
    distance_matrix = distance_matrix.get()
    return distance_matrix

def lcs_from_cosine_distance(matrix1, matrix2):
    # Вычисляем матрицу косинусных расстояний
    distance_matrix = gpu_cosine_distance_matrix(matrix1, matrix2)

    # Вычисляем среднее значение косинусного расстояния
    mean_distance = cp.mean(distance_matrix)

    # Устанавливаем порог как 25% от среднего значения
    threshold = mean_distance * 0.01
    print(threshold)
    # Преобразуем матрицу расстояний в матрицу совпадений
    match_matrix = (distance_matrix < threshold).astype(int)
    print(match_matrix)
    # Инициализируем DP таблицу
    rows, cols = match_matrix.shape
    dp = [[0] * (cols + 1) for _ in range(rows + 1)]

    # Заполняем DP таблицу
    for i in range(1, rows + 1):
        for j in range(1, cols + 1):
            if match_matrix[i-1][j-1] == 1:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])

    # Восстанавливаем LCS
    lcs = []
    i, j = rows, cols
    while i > 0 and j > 0:
        if match_matrix[i-1][j-1] == 1:
            lcs.append(matrix1[i-1])
            i -= 1
            j -= 1
        elif dp[i-1][j] > dp[i][j-1]:
            i -= 1
        else:
            j -= 1

    lcs.reverse()
    return lcs

# Пример использования:
if __name__ == "__main__":
    import numpy as np

    # Пример данных
    matrix1 = cp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    matrix2 = cp.array([ [4, 5, 6],[7, 8, 9], [10, 11, 12]])

    result = lcs_from_cosine_distance(matrix1, matrix2)

    # Преобразуем результат из cupy array в numpy array для вывода
    result_np = [cp.asnumpy(arr) for arr in result]

    print(result_np)  # Ожидается список numpy array с элементами LCS

0.00013616403823634208
[[0 0 0]
 [1 0 0]
 [0 1 0]]
[array([4, 5, 6]), array([7, 8, 9])]


In [ ]:
license_test = extract_frame_embeddings("lic.mp4")
piracy_test = extract_frame_embeddings("piracy.mp4")

In [ ]:
license_np_test = np.asarray(license_test, dtype="object")
piracy_np_test = np.asarray(piracy_test, dtype="object")

In [ ]:
print(piracy_np_test.shape)

In [ ]:
matrix_test = cosine_distance_matrix(license_np_test, piracy_np_test)

In [ ]:
print(matrix_test.shape)

In [ ]:
make_plt_rows(matrix_test)

In [ ]:
make_plt_columns(matrix_test)